<a href="https://colab.research.google.com/github/vZekii/MachineLearningAssignment2/blob/main/Machine_Learning_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task - Something something model or implementation

## Introduction
Discuss the definition, training and motivations of the model/study


I will do a combination of the 2 below tasks:
### Task 1
Implement a Perceptron 2-class classification model from scratch, ensuring that you cover all the essential components. This includes:

Score Computation: Implement the computation of the weighted sum of the inputs. You may want to investigate how to process multiple samples in one call of functions using `numpy`, `pytorch` libraries.

Classification Rule: Define the rule that maps the computed score to one of the two classes.

Parameter Updating Rule: Implement the rule for updating the weights and bias in response to the training data. (We will introduce this step shortly, but the implementation is straightforward, you can easily complete via help with ChatGPT/google)

Further Consideration: Pay particular attention to scenarios where the data is not linearly separable. Analyse how your implementation behaves in these cases and discuss any strategies you might use to handle them.

### Task 2
Parameter Sensitivity: Build upon the basic individual-sample update rule, we can consider summing up the update signals from multiple samples (a training batch), but each time with a certain step-size (learning rate, implemented in the given notebook). Study how the perceptron model behaves with different hyperparameters like learning rate and batch size. How do these parameters affect the speed and stability of learning?

# Task definition
I will implement a 2-class perceptron in python, including its functionalities of Score Computation, classification rules, parameter updating, while also analysing cases where data is not linearly seperable and what strategies can be used to overcome this

I have asked chat gpt to generate some initial code for this task

1. Import Required Libraries:
Start by importing the necessary libraries, including scikit-learn to load the Iris dataset and NumPy for numerical operations.

In [23]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split


2. Load and Prepare the Iris Dataset:
Load the Iris dataset and split it into training and testing sets.

In [24]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


3. Implement the Multi-Class Perceptron:
Create a class for the multi-class perceptron. The perceptron will have weights and biases for each class.

In [25]:
class MultiClassPerceptron:
    def __init__(self, num_classes, num_features, learning_rate=0.1, num_epochs=100):
        self.num_classes = num_classes
        self.num_features = num_features
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.weights = np.zeros((num_classes, num_features))
        self.biases = np.zeros(num_classes)

    def predict(self, x):
        scores = np.dot(self.weights, x) + self.biases
        return np.argmax(scores)

    def train(self, X, y):
        for epoch in range(self.num_epochs):
            for i in range(len(X)):
                x = X[i]
                y_true = y[i]

                y_pred = self.predict(x)
                if y_pred != y_true:
                    self.weights[y_true] += self.learning_rate * x
                    self.biases[y_true] += self.learning_rate
                    self.weights[y_pred] -= self.learning_rate * x
                    self.biases[y_pred] -= self.learning_rate

    def train_batch(self, X, y):
      """
      chatgpt implementation of a training batch
      """
      for epoch in range(self.num_epochs):
          # Initialize updates for weights and biases
          weight_updates = np.zeros(self.weights.shape)
          bias_updates = np.zeros(self.biases.shape)

          for i in range(len(X)):
              x = X[i]
              y_true = y[i]

              y_pred = self.predict(x)
              if y_pred != y_true:
                  weight_updates[y_true] += x
                  bias_updates[y_true] += 1
                  weight_updates[y_pred] -= x
                  bias_updates[y_pred] -= 1

          # Update weights and biases with accumulated updates
          self.weights += self.learning_rate * weight_updates
          self.biases += self.learning_rate * bias_updates

4. Train the Multi-Class Perceptron:
Instantiate the multi-class perceptron, and train it on the training data.

In [26]:
num_classes = len(np.unique(y_train))
num_features = X_train.shape[1]
perceptron = MultiClassPerceptron(num_classes, num_features)
train_multi = True

if not train_multi:
  # Use standard single-updating method
  perceptron.train(X_train, y_train)

else:
  # Modify your training loop to use batches
  batch_size = 10  # Adjust the batch size as needed
  num_epochs = 100

  for epoch in range(num_epochs):
      for i in range(0, len(X_train), batch_size):
          X_batch = X_train[i:i+batch_size]
          y_batch = y_train[i:i+batch_size]

          perceptron.train_batch(X_batch, y_batch)


5. Evaluate the Model:
Test the model on the testing data and calculate the accuracy.

In [27]:
standard_algo = False
if standard_algo:
  correct = 0
  for i in range(len(X_test)):
      if perceptron.predict(X_test[i]) == y_test[i]:
          correct += 1

  accuracy = correct / len(X_test)
  print(f"Accuracy: {accuracy * 100:.2f}%")

else:
  # Implement hyperparameter optimisation
  learning_rates = [0.1, 0.01, 0.001]
  batch_sizes = [10, 32, 64]
  num_epochs = [50, 100, 200]
  best_accuracy = 0
  best_hyperparameters = {}

  for lr in learning_rates:
      for batch_size in batch_sizes:
          for epochs in num_epochs:
              perceptron = MultiClassPerceptron(num_classes, num_features, learning_rate=lr, num_epochs=epochs)

              for epoch in range(epochs):
                  for i in range(0, len(X_train), batch_size):
                      X_batch = X_train[i:i+batch_size]
                      y_batch = y_train[i:i+batch_size]

                      perceptron.train_batch(X_batch, y_batch)

              correct = 0
              for i in range(len(X_test)):
                  if perceptron.predict(X_test[i]) == y_test[i]:
                      correct += 1

              accuracy = correct / len(X_test)

              if accuracy > best_accuracy:
                  best_accuracy = accuracy
                  best_hyperparameters = {'learning_rate': lr, 'batch_size': batch_size, 'num_epochs': epochs}

  print(f"Best Hyperparameters: {best_hyperparameters}")
  print(f"Best Accuracy: {best_accuracy * 100:.2f}%")

Best Hyperparameters: {'learning_rate': 0.1, 'batch_size': 64, 'num_epochs': 50}
Best Accuracy: 100.00%
